# Тестовое задание Авито

In [2]:
import re
import unicodedata
from typing import List, Tuple, Dict
import random
import numpy as np
import json
import torch
from pathlib import Path

In [3]:
RANDOM_STATE = 99

random.seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)
torch.manual_seed(RANDOM_STATE)
torch.cuda.manual_seed(RANDOM_STATE)
torch.cuda.manual_seed_all(RANDOM_STATE)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

## Создание датасета

Код ниже выполняет следующие действия:
1. Загружает строки из файла ./dataset_raw/articles.txt.
2. Очищает каждую строку (Unicode-нормализация, удаление control chars, удаление эмодзи, нормализация пробелов).
3. Разбивает на короткие сегменты (2–8 слов).
4. Для каждого сегмента строит:
    - вход (строка без пробелов),
    - список меток (1 после символа, если там был пробел).
5. Собирает словарь char2id (кириллица, латиница, цифры, знаки).
6. Возвращает готовый датасет: X (индексы символов), y (метки), char2id.


Для обучения модели используется файл articles.txt, взятый из датасета Complex Russian Dataset (https://www.kaggle.com/datasets/artalmaz31/complex-russian-dataset). Данный файл содержит статьи на разные темы, опубликованные на dzen.ru. Для обучения модели был взят именно этот набор текстов, так как он содержит тексты на русском языке с иногда встречающимися английскими названиями и числами, что по структуре напоминает тестовые данные.

In [34]:
def read_corpus(path: str) -> List[str]:
    """
    Загружает корпус текстов из файла.
    
    :param path: путь к файлу (каждая строка = статья или абзац).
    :return: список строк
    """
    with open(path, "r", encoding="utf-8") as f:
        lines = [line.strip() for line in f if line.strip()]
    return lines

In [35]:
def clean_text(text: str) -> str:
    """
    Очищает текст:
    - нормализует Unicode (NFC)
    - удаляет управляющие символы
    - нормализует пробелы
    - удаляет эмодзи
    
    :param text: исходная строка
    :return: очищенный текст
    """
    # Unicode нормализация
    text = unicodedata.normalize("NFC", text)
    
    # Убираем control chars
    text = "".join(ch for ch in text if unicodedata.category(ch)[0] != "C")
    
    # Убираем эмодзи (символы за пределами Basic Multilingual Plane и спец.категории)
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # смайлы
        u"\U0001F300-\U0001F5FF"  # пиктограммы
        u"\U0001F680-\U0001F6FF"  # транспорт
        u"\U0001F1E0-\U0001F1FF"  # флаги
        u"\U00002700-\U000027BF"  # символы-стрелки
        u"\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE)
    text = emoji_pattern.sub("", text)
    
    # Нормализация пробелов
    text = re.sub(r"\s+", " ", text).strip()
    
    return text

In [36]:
def split_into_segments(text: str, min_words: int = 2, max_words: int = 8) -> List[str]:
    """
    Разбивает текст на короткие сегменты (min_words-max_words слов).
    
    :param text: очищенный текст
    :param min_words: минимум слов в сегменте
    :param max_words: максимум слов в сегменте
    :return: список коротких сегментов
    """
    words = text.split()
    segments = []
    
    if len(words) < min_words:
        return []
    
    # Берем окна разных длин
    for start in range(len(words)):
        for length in range(min_words, max_words + 1):
            end = start + length
            if end <= len(words):
                segment = " ".join(words[start:end])
                segments.append(segment)
    return segments

In [37]:
def generate_no_space_input(segment: str) -> Tuple[str, List[int]]:
    """
    Генерирует вход без пробелов и метки (0/1 после каждого символа).
    
    :param segment: строка с пробелами
    :return: (вход без пробелов, список меток)
    """
    input_text = segment.replace(" ", "")
    labels = []
    i = 0
    for word in segment.split():
        for j, ch in enumerate(word):
            # Если символ последний в слове, ставим 1, иначе 0
            if j == len(word) - 1 and i < len(input_text):
                labels.append(1)
            else:
                labels.append(0)
            i += 1
    # Последний символ не должен иметь пробела после себя
    if labels:
        labels[-1] = 0
    return input_text, labels

In [38]:
def build_char2id(texts: List[str]) -> Dict[str, int]:
    """
    Строит словарь символов для корпуса.
    
    :param texts: список строк (inputs без пробелов)
    :return: словарь {символ: id}
    """
    chars = set("".join(texts))
    sorted_chars = sorted(chars)
    char2id = {ch: idx + 1 for idx, ch in enumerate(sorted_chars)}  # Нумерация с 1
    char2id["<UNK>"] = len(char2id) + 1  # Для неизвестных символов
    return char2id

In [39]:
def build_dataset(path: str, max_segments: int = 10000) -> Tuple[List[List[int]], List[List[int]], Dict[str, int]]:
    """
    Основная функция: строит датасет из корпуса статей.
    
    :param path: путь к файлу с текстами
    :param max_segments: максимум сегментов в датасете
    :return: (X, y, char2id)
        X — список последовательностей индексов символов (inputs)
        y — список меток (0/1 для каждого символа)
        char2id — словарь символов
    """
    lines = read_corpus(path)
    
    # Очистка + сегментация
    all_segments = []
    for line in lines:
        cleaned = clean_text(line)
        segs = split_into_segments(cleaned)
        all_segments.extend(segs)
    
    # Перемешиваем и берем первые max_segments
    random.shuffle(all_segments)
    all_segments = all_segments[:max_segments]
    
    # Генерация данных
    inputs, labels = [], []
    for seg in all_segments:
        inp, lab = generate_no_space_input(seg)
        if inp and lab:
            inputs.append(inp)
            labels.append(lab)
    
    # Строим словарь
    char2id = build_char2id(inputs)
    
    # Преобразуем inputs в индексы
    X = []
    for text in inputs:
        seq = [char2id.get(ch, char2id["<UNK>"]) for ch in text]
        X.append(seq)
    
    return X, labels, char2id

In [40]:
def save_dataset(X, y, char2id, out_dir="dataset"):
    """
    Сохраняет датасет и словарь для будущего использования.

    Args:
        X (np.ndarray): входные данные (последовательности id символов)
        y (np.ndarray): целевые метки (бинарные маски для пробелов)
        char2id (dict): словарь символ → id
        out_dir (str): путь к директории для сохранения
    """
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)

    # Сохраняем X и y
    np.save(out_dir / "X.npy", np.array(X, dtype=object))
    np.save(out_dir / "y.npy", np.array(y, dtype=object))

    # Сохраняем словарь
    with open(out_dir / "char2id.json", "w", encoding="utf-8") as f:
        json.dump(char2id, f, ensure_ascii=False, indent=2)

    print(f"Датасет сохранён в {out_dir}")

In [4]:
def load_dataset(out_dir="dataset"):
    """
    Загружает ранее сохранённый датасет и словарь.

    Args:
        out_dir (str): путь к директории

    Returns:
        X (np.ndarray), y (np.ndarray), char2id (dict)
    """
    out_dir = Path(out_dir)

    X = np.load(out_dir / "X.npy", allow_pickle=True)
    y = np.load(out_dir / "y.npy", allow_pickle=True)

    with open(out_dir / "char2id.json", "r", encoding="utf-8") as f:
        char2id = json.load(f)

    print(f"Датасет загружен из {out_dir}")
    return X, y, char2id

In [42]:
dataset_path = "./dataset_raw/articles.txt"  # файл со статьями
X, y, char2id = build_dataset(dataset_path, max_segments=5000)

In [43]:
save_dataset(X, y, char2id, out_dir="./dataset_processed")

Датасет сохранён в dataset_processed


In [5]:
X, y, char2id = load_dataset("./dataset_processed")

Датасет загружен из dataset_processed


In [6]:
print("Пример входа (индексы):", X[0])
print("Пример меток:", y[0])
print("Пример сегмента:", "".join([list(char2id.keys())[list(char2id.values()).index(idx)] for idx in X[0]]))
print("Размер словаря:", len(char2id))

Пример входа (индексы): [146, 135, 131, 129, 134, 128, 136, 140, 117, 122, 128, 136, 140, 141, 122, 131, 118, 131, 126, 135, 125, 134, 145, 135, 131, 128, 145, 127, 131, 127, 131, 130, 134, 125, 128, 122, 133, 131, 129, 125]
Пример меток: [0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
Пример сегмента: этомслучаелучшеобойтисьтолькоконсилероми
Размер словаря: 160
